# 2. Data Cleaning

In [ ]:
# imports
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from math import sqrt
from scipy.stats import norm

from IPython import display

import re
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_rows", None)

In [ ]:
# read in and assign dataframes
spray_df = pd.read_csv('datasets/spray.csv')
weather_df = pd.read_csv('datasets/weather.csv')
train_df = pd.read_csv('datasets/train.csv')
test_df = pd.read_csv('datasets/test.csv')

In [ ]:
#display the number of rows and columns
print(f'spray_df: {spray_df.shape}')
print(f'weather_df: {weather_df.shape}')
print(f'train_df: {train_df.shape}')
print(f'test_df: {test_df.shape}')

spray_df: (14835, 4)
weather_df: (2944, 22)
train_df: (10506, 12)
test_df: (116293, 11)


## train_df

In [ ]:
# check for observable missing values
train_df.isnull().sum()

Date                      0
Address                   0
Species                   0
Block                     0
Street                    0
Trap                      0
AddressNumberAndStreet    0
Latitude                  0
Longitude                 0
AddressAccuracy           0
NumMosquitos              0
WnvPresent                0
dtype: int64

In [ ]:
# check column values
for col in train_df:
    print(train_df[col].unique())

['2007-05-29' '2007-06-05' '2007-06-26' '2007-06-29' '2007-07-02'
 '2007-07-11' '2007-07-18' '2007-07-19' '2007-07-25' '2007-07-27'
 '2007-08-01' '2007-08-02' '2007-08-03' '2007-08-07' '2007-08-08'
 '2007-08-09' '2007-08-15' '2007-08-16' '2007-08-17' '2007-08-21'
 '2007-08-22' '2007-08-24' '2007-08-28' '2007-09-04' '2007-09-05'
 '2007-09-06' '2007-09-12' '2007-09-18' '2007-09-19' '2007-09-24'
 '2007-09-25' '2007-10-04' '2007-10-09' '2009-05-28' '2009-06-02'
 '2009-06-03' '2009-06-05' '2009-06-12' '2009-06-15' '2009-06-19'
 '2009-06-22' '2009-06-26' '2009-06-29' '2009-07-06' '2009-07-10'
 '2009-07-13' '2009-07-17' '2009-07-24' '2009-07-27' '2009-07-31'
 '2009-08-07' '2009-08-13' '2009-08-25' '2009-08-27' '2009-09-03'
 '2009-09-14' '2009-09-17' '2009-09-25' '2009-10-01' '2011-06-10'
 '2011-06-17' '2011-06-24' '2011-06-30' '2011-07-11' '2011-07-15'
 '2011-07-25' '2011-07-29' '2011-08-05' '2011-08-12' '2011-08-19'
 '2011-08-26' '2011-09-01' '2011-09-02' '2011-09-12' '2011-09-16'
 '2011-09-

No missing values or non-unicode charaters present in train_df. However, plenty of uppercase letters can be observed in the dataframe.

In [ ]:
# define function for lowercase letters
def clean_columns(col):
    return col.lower()

In [ ]:
# apply lowercase to all columns in train_df
train_df.columns = [clean_columns(col) for col in train_df]

In [ ]:
# check for duplicate rows in train_df
print(f'There are {len(train_df[train_df.duplicated()])} duplicate rows in train_df.')

There are 813 duplicate rows in train_df.


In [ ]:
# removing duplicates
train_df.drop_duplicates(inplace=True)

In [ ]:
# check if duplicates are removed
print(f'There are {len(train_df[train_df.duplicated()])} duplicate rows in train_df.')

There are 0 duplicate rows in train_df.


In [ ]:
train_df.head(1)

,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


## weather_df

In [ ]:
#output sample size of weather dataframe
weather_df.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [ ]:
#understand what are the features in weather dataframe
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Station      2944 non-null   int64  
 1   Date         2944 non-null   object 
 2   Tmax         2944 non-null   int64  
 3   Tmin         2944 non-null   int64  
 4   Tavg         2944 non-null   object 
 5   Depart       2944 non-null   object 
 6   DewPoint     2944 non-null   int64  
 7   WetBulb      2944 non-null   object 
 8   Heat         2944 non-null   object 
 9   Cool         2944 non-null   object 
 10  Sunrise      2944 non-null   object 
 11  Sunset       2944 non-null   object 
 12  CodeSum      2944 non-null   object 
 13  Depth        2944 non-null   object 
 14  Water1       2944 non-null   object 
 15  SnowFall     2944 non-null   object 
 16  PrecipTotal  2944 non-null   object 
 17  StnPressure  2944 non-null   object 
 18  SeaLevel     2944 non-null   object 
 19  Result

In [ ]:
# apply lowercase to all columns in weather_df
weather_df.columns = [clean_columns(col) for col in weather_df]

In [ ]:
#output sample of updated weather dataframe
weather_df.head()

,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,codesum,depth,water1,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [ ]:
#see the number of rows and columns for weather dataframe
weather_df.shape

(2944, 22)

In [ ]:
# check for missing data

total_missing_stn1 = 0
total_missing_stn2 = 0

for col in weather_df[weather_df.columns]:
    
    station_1 = len(weather_df[(weather_df[col].isin(['M','-'])) & (weather_df.station==1)])
    total_missing_stn1 = total_missing_stn1 + station_1
    print(col + ' has ' + str(station_1) + ' missing values at station 1')
    
    station_2 = len(weather_df[(weather_df[col].isin(['M','-'])) & (weather_df.station==2)])
    total_missing_stn2 = total_missing_stn2 + station_2
    print(col + ' has ' + str(station_2) + ' missing values at station 2')
    print()

print("Station 1 has a total of ", total_missing_stn1, " missing data values")
print("Station 2 has a total of ", total_missing_stn2, " missing data values")

station has 0 missing values at station 1
station has 0 missing values at station 2

date has 0 missing values at station 1
date has 0 missing values at station 2

tmax has 0 missing values at station 1
tmax has 0 missing values at station 2

tmin has 0 missing values at station 1
tmin has 0 missing values at station 2

tavg has 0 missing values at station 1
tavg has 11 missing values at station 2

depart has 0 missing values at station 1
depart has 1472 missing values at station 2

dewpoint has 0 missing values at station 1
dewpoint has 0 missing values at station 2

wetbulb has 3 missing values at station 1
wetbulb has 1 missing values at station 2

heat has 0 missing values at station 1
heat has 11 missing values at station 2

cool has 0 missing values at station 1
cool has 11 missing values at station 2

sunrise has 0 missing values at station 1
sunrise has 1472 missing values at station 2

sunset has 0 missing values at station 1
sunset has 1472 missing values at station 2

codesu

Station 1 has a total of 1474 missing data values and station 2 has a total of  4424  missing data value. Therefore, we will only be making use of data from station 1 for analysis since both weather stations collect the same measurement features over the same duration for the city of Chicago.

In [ ]:
# redefining weather_df for only station 1
weather_df = weather_df[weather_df["station"]==1]
weather_df.head()

,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,codesum,depth,water1,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9
6,1,2007-05-04,66,49,58,4,41,50,7,0,...,RA,0,M,0.0,T,29.31,30.05,10.4,8,10.8
8,1,2007-05-05,66,53,60,5,38,49,5,0,...,,0,M,0.0,T,29.40,30.10,11.7,7,12.0


In [ ]:
#see the number of rows and columns of the updated weather dataframe
weather_df.shape

(1472, 22)

In [ ]:
# determine number of 'M' values (missing data) in each column
weather_df.isin(['M']).sum()[weather_df.isin(['M']).sum() > 0].sort_values(ascending=False)

water1         1472
sealevel          5
wetbulb           3
stnpressure       2
dtype: int64

In [ ]:
# drop columns that are not needed
weather_df.drop(['tmax', 'tmin', 'depart', 'heat', 'cool', 'sunrise', 'sunset'], axis=1, inplace=True)

In [ ]:
#see sample output of the new weather dataframe
weather_df.head()

,station,date,tavg,dewpoint,wetbulb,codesum,depth,water1,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2007-05-01,67,51,56,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
2,1,2007-05-02,51,42,47,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
4,1,2007-05-03,56,40,48,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9
6,1,2007-05-04,58,41,50,RA,0,M,0.0,T,29.31,30.05,10.4,8,10.8
8,1,2007-05-05,60,38,49,,0,M,0.0,T,29.40,30.10,11.7,7,12.0


In [ ]:
# remove rows with 'M' values (missing data) for the 'wetbulb' column
weather_df = weather_df[weather_df.wetbulb != 'M']

# check for the number of missing values in the 'wetbulb' column
weather_df['wetbulb'].isin(['M']).sum()

0

In [ ]:
# check for unique values in 'codesum' column
weather_df['codesum'].unique()

array([' ', 'BR', 'RA', 'BR HZ', 'RA BR', 'RA VCTS', 'RA HZ',
       'TSRA RA BR HZ', 'HZ', 'TSRA BR HZ', 'TSRA RA BR', 'TSRA RA HZ',
       'TSRA BR HZ VCTS', 'TSRA', 'TSRA BR HZ FU', 'BR HZ FU',
       'TSRA RA VCTS', 'HZ VCTS', 'RA BR VCTS', 'TSRA RA BR VCTS',
       'DZ BR', 'TS TSRA RA BR HZ', 'RA BR HZ', 'TSRA RA',
       'TS TSRA RA BR', 'TS RA BR', 'TS TSRA RA', 'TS TSRA RA BR VCTS',
       'TS RA', 'TSRA BR', 'RA DZ FG+ BCFG BR', 'RA FG+ MIFG BR', 'RA DZ',
       'RA DZ BR', 'TSRA RA FG+ FG BR', 'TSRA DZ FG+ FG BR HZ', 'TS TSRA',
       'TSRA HZ', 'TS', 'FG+ BR HZ', 'TS TSRA BR HZ', 'DZ BR HZ',
       'TS TSRA BR', 'RA BR FU', 'TS BR HZ', 'FG+ BR', 'FG+ MIFG BR',
       'TSRA RA FG BR', 'DZ', 'RA DZ BR HZ', 'TS BR', 'TS RA FG+ FG BR',
       'RA DZ FG+ FG BR', 'RA BR SQ', 'RA FG BR', 'RA BR HZ FU',
       'MIFG BCFG BR', 'TSRA RA FG+ FG BR HZ', 'TSRA DZ BR HZ',
       'TS TSRA RA FG BR HZ', 'RA DZ FG+ BR', 'TSRA FG+ BR HZ',
       'FG+ FG BR HZ', 'FG BR HZ', 'TS RA BR HZ', 'BC

In [ ]:
# define function to remove whitespace from text
def clean_text(text):
    text = re.sub(' ', '', text)
    return text

In [ ]:
# remove whitespace in values in 'codesum' column to check for cells that are empty ("NO SIGN" = Moderate according to NOAA data documentation) 
weather_df["codesum"].map(clean_text)

0                      
2                    BR
4                      
6                    RA
8                      
10                     
12                   RA
14                   BR
16                 BRHZ
18                   BR
20                     
22                     
24                     
26                     
28                 RABR
30                 RABR
32                     
34                     
36                     
38               RAVCTS
40                     
42                     
44                     
46                 RAHZ
48                     
50           TSRARABRHZ
52                 RABR
54                   RA
56                 BRHZ
58                   HZ
60                 RAHZ
62             TSRABRHZ
64                 RABR
66           TSRARABRHZ
68             TSRARABR
70                 RABR
72                   RA
74                     
76                     
78                     
80                     
82              

In [ ]:
# replace empty string values with "moderate" according to NOAA data documentation
weather_df["codesum"] = weather_df["codesum"].replace(" ","moderate")

In [ ]:
# check that "moderate" has been filled in
weather_df["codesum"].unique()

array(['moderate', 'BR', 'RA', 'BR HZ', 'RA BR', 'RA VCTS', 'RA HZ',
       'TSRA RA BR HZ', 'HZ', 'TSRA BR HZ', 'TSRA RA BR', 'TSRA RA HZ',
       'TSRA BR HZ VCTS', 'TSRA', 'TSRA BR HZ FU', 'BR HZ FU',
       'TSRA RA VCTS', 'HZ VCTS', 'RA BR VCTS', 'TSRA RA BR VCTS',
       'DZ BR', 'TS TSRA RA BR HZ', 'RA BR HZ', 'TSRA RA',
       'TS TSRA RA BR', 'TS RA BR', 'TS TSRA RA', 'TS TSRA RA BR VCTS',
       'TS RA', 'TSRA BR', 'RA DZ FG+ BCFG BR', 'RA FG+ MIFG BR', 'RA DZ',
       'RA DZ BR', 'TSRA RA FG+ FG BR', 'TSRA DZ FG+ FG BR HZ', 'TS TSRA',
       'TSRA HZ', 'TS', 'FG+ BR HZ', 'TS TSRA BR HZ', 'DZ BR HZ',
       'TS TSRA BR', 'RA BR FU', 'TS BR HZ', 'FG+ BR', 'FG+ MIFG BR',
       'TSRA RA FG BR', 'DZ', 'RA DZ BR HZ', 'TS BR', 'TS RA FG+ FG BR',
       'RA DZ FG+ FG BR', 'RA BR SQ', 'RA FG BR', 'RA BR HZ FU',
       'MIFG BCFG BR', 'TSRA RA FG+ FG BR HZ', 'TSRA DZ BR HZ',
       'TS TSRA RA FG BR HZ', 'RA DZ FG+ BR', 'TSRA FG+ BR HZ',
       'FG+ FG BR HZ', 'FG BR HZ', 'TS RA BR H

In [ ]:
#see sample output of the updated weather dataframe
weather_df.head()

,station,date,tavg,dewpoint,wetbulb,codesum,depth,water1,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2007-05-01,67,51,56,moderate,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
2,1,2007-05-02,51,42,47,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
4,1,2007-05-03,56,40,48,moderate,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9
6,1,2007-05-04,58,41,50,RA,0,M,0.0,T,29.31,30.05,10.4,8,10.8
8,1,2007-05-05,60,38,49,moderate,0,M,0.0,T,29.40,30.10,11.7,7,12.0


In [ ]:
# check for the the number of '0' values in depth column
weather_df['depth'].value_counts()

0    1469
Name: depth, dtype: int64

In [ ]:
# we can drop the 'depth' since all the rows have the value '0'
weather_df.drop(['depth'], axis=1, inplace=True)

In [ ]:
# check for unique values in 'water1' column
weather_df['water1'].unique()

array(['M'], dtype=object)

In [ ]:
# since the entire column consists of 'M' (missing values), we will drop the 'water1' column as well
weather_df.drop(['water1'], axis=1, inplace=True)

In [ ]:
# check for values counts in 'snowfall' column
weather_df['snowfall'].value_counts()

0.0    1456
  T      12
0.1       1
Name: snowfall, dtype: int64

In [ ]:
# since data was collected between May-Oct (not during winter season),
# more than half the data is missing and just under half are 0, we drop the whole column
weather_df.drop(['snowfall'], axis=1, inplace=True)

In [ ]:
# check for unique values in 'preciptotal' column
weather_df["preciptotal"].unique()

array(['0.00', '  T', '0.13', '0.38', '0.14', '0.07', '0.02', '1.01',
       '0.04', '0.01', '0.19', '0.39', '0.42', '0.28', '0.58', '0.05',
       '0.16', '0.15', '0.40', '0.30', '0.43', '0.11', '1.55', '0.17',
       '0.03', '1.43', '0.26', '1.31', '0.06', '0.29', '0.41', '0.23',
       '0.09', '1.33', '0.91', '0.37', '2.35', '0.24', '0.34', '0.18',
       '0.36', '0.27', '0.33', '1.28', '0.76', '0.95', '0.08', '0.12',
       '0.48', '0.52', '0.64', '1.24', '0.20', '0.72', '0.73', '1.61',
       '0.74', '1.05', '2.43', '2.90', '1.23', '6.64', '1.44', '0.82',
       '0.80', '0.32', '0.67', '0.55', '1.73', '0.44', '0.45', '3.97',
       '0.25', '0.68', '0.63', '0.35', '0.93', '0.22', '0.75', '0.89',
       '0.21', '0.46', '1.03', '0.60', '0.81', '0.97', '1.49', '1.02',
       '0.59', '0.87', '0.85', '0.54', '0.79', '2.79', '3.64', '0.10',
       '0.31', '2.24', '1.06', '1.88', '6.86', '0.71', '0.70', '1.11',
       '0.65', '1.21', '0.77', '1.34', '0.47', '1.12', '0.57', '0.51',
       

In [ ]:
# according to  NOAA documentation, T stands for Trace amount of precipitation
# replace T with 0.005 (NEED TO COME BACK TO THIS)
weather_df["preciptotal"] = weather_df["preciptotal"].replace("  T","0.005")

In [ ]:
# check that all T replaced with 0.005
weather_df["preciptotal"].unique()

array(['0.00', '0.005', '0.13', '0.38', '0.14', '0.07', '0.02', '1.01',
       '0.04', '0.01', '0.19', '0.39', '0.42', '0.28', '0.58', '0.05',
       '0.16', '0.15', '0.40', '0.30', '0.43', '0.11', '1.55', '0.17',
       '0.03', '1.43', '0.26', '1.31', '0.06', '0.29', '0.41', '0.23',
       '0.09', '1.33', '0.91', '0.37', '2.35', '0.24', '0.34', '0.18',
       '0.36', '0.27', '0.33', '1.28', '0.76', '0.95', '0.08', '0.12',
       '0.48', '0.52', '0.64', '1.24', '0.20', '0.72', '0.73', '1.61',
       '0.74', '1.05', '2.43', '2.90', '1.23', '6.64', '1.44', '0.82',
       '0.80', '0.32', '0.67', '0.55', '1.73', '0.44', '0.45', '3.97',
       '0.25', '0.68', '0.63', '0.35', '0.93', '0.22', '0.75', '0.89',
       '0.21', '0.46', '1.03', '0.60', '0.81', '0.97', '1.49', '1.02',
       '0.59', '0.87', '0.85', '0.54', '0.79', '2.79', '3.64', '0.10',
       '0.31', '2.24', '1.06', '1.88', '6.86', '0.71', '0.70', '1.11',
       '0.65', '1.21', '0.77', '1.34', '0.47', '1.12', '0.57', '0.51',
     

In [ ]:
# redefine weather_df without the rows with missing values ('M') in 'preciptotal' column
weather_df = weather_df[weather_df.preciptotal != 'M']
weather_df = weather_df[weather_df.stnpressure != 'M']
weather_df = weather_df[weather_df.sealevel != 'M']

In [ ]:
# check that there are no more missing values for the 'preciptotal' column
weather_df['preciptotal'].isin(['M']).sum()

0

In [ ]:
# check that there are no more missing values for the 'stnpressure' column
weather_df['stnpressure'].isin(['M']).sum()

0

In [ ]:
# check that there are no more missing values for the 'sealevel' column
weather_df['sealevel'].isin(['M']).sum()

0

In [ ]:
#see sample output of the updated weather dataframe
weather_df.head()

,station,date,tavg,dewpoint,wetbulb,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2007-05-01,67,51,56,moderate,0.00,29.10,29.82,1.7,27,9.2
2,1,2007-05-02,51,42,47,BR,0.00,29.38,30.09,13.0,4,13.4
4,1,2007-05-03,56,40,48,moderate,0.00,29.39,30.12,11.7,7,11.9
6,1,2007-05-04,58,41,50,RA,0.005,29.31,30.05,10.4,8,10.8
8,1,2007-05-05,60,38,49,moderate,0.005,29.40,30.10,11.7,7,12.0


In [ ]:
#see information of the updated weather dataframe
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1464 entries, 0 to 2942
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   station      1464 non-null   int64  
 1   date         1464 non-null   object 
 2   tavg         1464 non-null   object 
 3   dewpoint     1464 non-null   int64  
 4   wetbulb      1464 non-null   object 
 5   codesum      1464 non-null   object 
 6   preciptotal  1464 non-null   object 
 7   stnpressure  1464 non-null   object 
 8   sealevel     1464 non-null   object 
 9   resultspeed  1464 non-null   float64
 10  resultdir    1464 non-null   int64  
 11  avgspeed     1464 non-null   object 
dtypes: float64(1), int64(3), object(8)
memory usage: 148.7+ KB


In [ ]:
# convert object data types to int and float
weather_df['tavg'] = weather_df['tavg'].astype(int)
weather_df['wetbulb'] = weather_df['wetbulb'].astype(int)
weather_df["preciptotal"] = weather_df["preciptotal"].astype(float)
weather_df['stnpressure'] = weather_df['stnpressure'].astype(float)
weather_df['sealevel'] = weather_df['sealevel'].astype(float)
weather_df['avgspeed'] = weather_df['avgspeed'].astype(float)

In [ ]:
#see information of the updated weather dataframe
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1464 entries, 0 to 2942
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   station      1464 non-null   int64  
 1   date         1464 non-null   object 
 2   tavg         1464 non-null   int64  
 3   dewpoint     1464 non-null   int64  
 4   wetbulb      1464 non-null   int64  
 5   codesum      1464 non-null   object 
 6   preciptotal  1464 non-null   float64
 7   stnpressure  1464 non-null   float64
 8   sealevel     1464 non-null   float64
 9   resultspeed  1464 non-null   float64
 10  resultdir    1464 non-null   int64  
 11  avgspeed     1464 non-null   float64
dtypes: float64(5), int64(5), object(2)
memory usage: 148.7+ KB


In [ ]:
#see the number of rows and columns of the updated weather dataframe
weather_df.shape

(1464, 12)

In [ ]:
#see sample output of the updated weather dataframe
weather_df.head()

,station,date,tavg,dewpoint,wetbulb,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2007-05-01,67,51,56,moderate,0.000,29.10,29.82,1.7,27,9.2
2,1,2007-05-02,51,42,47,BR,0.000,29.38,30.09,13.0,4,13.4
4,1,2007-05-03,56,40,48,moderate,0.000,29.39,30.12,11.7,7,11.9
6,1,2007-05-04,58,41,50,RA,0.005,29.31,30.05,10.4,8,10.8
8,1,2007-05-05,60,38,49,moderate,0.005,29.40,30.10,11.7,7,12.0


## combined_df

In [ ]:
#see the number of rows and columns of the train and weather dataframe before combining
print(f'train_df: {train_df.shape}')
print(f'weather_df: {weather_df.shape}')

train_df: (9693, 12)
weather_df: (1464, 12)


There are 9693 rows in train_df and 1464 rows in weather_df after removing station 2. As there are duplicated dates in train_df for the mosquito species column, doing an inner merge will cause a huge loss of data (from 9k rows to 1.4k row). 

Therefore, we will perform an outer merge, followed by dropping null values caused by the difference in rows.

In [ ]:
# outer merge train_df and weather_df
combined_df = pd.merge(left=train_df,right=weather_df,on="date",how="outer")

In [ ]:
# check number of null values in combined_df
combined_df.isnull().sum()

date                         0
address                   1370
species                   1370
block                     1370
street                    1370
trap                      1370
addressnumberandstreet    1370
latitude                  1370
longitude                 1370
addressaccuracy           1370
nummosquitos              1370
wnvpresent                1370
station                     92
tavg                        92
dewpoint                    92
wetbulb                     92
codesum                     92
preciptotal                 92
stnpressure                 92
sealevel                    92
resultspeed                 92
resultdir                   92
avgspeed                    92
dtype: int64

In [ ]:
# drop rows with null values
combined_df = combined_df.dropna()

In [ ]:
# double check for null values in combined_df 
combined_df.isnull().sum()

date                      0
address                   0
species                   0
block                     0
street                    0
trap                      0
addressnumberandstreet    0
latitude                  0
longitude                 0
addressaccuracy           0
nummosquitos              0
wnvpresent                0
station                   0
tavg                      0
dewpoint                  0
wetbulb                   0
codesum                   0
preciptotal               0
stnpressure               0
sealevel                  0
resultspeed               0
resultdir                 0
avgspeed                  0
dtype: int64

In [ ]:
#see sample output of the updated weather dataframe
combined_df.head(1)

,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,...,tavg,dewpoint,wetbulb,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9.0,...,74.0,58.0,65.0,BR HZ,0.0,29.39,30.11,5.8,18.0,6.5


In [ ]:
# since the combined_df only consists of results from station 1, we can drop the 'station' column
# additionally, to simplify our analysis, we are dropping repeated columns for the adddress.
combined_df.drop(['station', 'address', 'addressnumberandstreet'], axis = 1, inplace = True)

In [ ]:
# display columns left in combined_df
combined_df.columns

Index(['date', 'species', 'block', 'street', 'trap', 'latitude', 'longitude',
       'addressaccuracy', 'nummosquitos', 'wnvpresent', 'tavg', 'dewpoint',
       'wetbulb', 'codesum', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed'],
      dtype='object')

In [ ]:
# cleaning 'date' column by converting to timestamp
combined_df['date'] = pd.to_datetime(combined_df['date'])

# create 3 new columns to separate to year, month and day
combined_df['year'] = combined_df['date'].apply(lambda time: time.year)
combined_df['month'] = combined_df['date'].apply(lambda time: time.month)
combined_df['day'] = combined_df['date'].apply(lambda time: time.day)

In [ ]:
#see a sample output of the updated weather dataframe
combined_df.head(1)

,date,species,block,street,trap,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent,...,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day
0,2007-05-29,CULEX PIPIENS/RESTUANS,41.0,N OAK PARK AVE,T002,41.95469,-87.800991,9.0,1.0,0.0,...,BR HZ,0.0,29.39,30.11,5.8,18.0,6.5,2007,5,29


In [ ]:
#showing information of the combined dataframe
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9601 entries, 0 to 9692
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             9601 non-null   datetime64[ns]
 1   species          9601 non-null   object        
 2   block            9601 non-null   float64       
 3   street           9601 non-null   object        
 4   trap             9601 non-null   object        
 5   latitude         9601 non-null   float64       
 6   longitude        9601 non-null   float64       
 7   addressaccuracy  9601 non-null   float64       
 8   nummosquitos     9601 non-null   float64       
 9   wnvpresent       9601 non-null   float64       
 10  tavg             9601 non-null   float64       
 11  dewpoint         9601 non-null   float64       
 12  wetbulb          9601 non-null   float64       
 13  codesum          9601 non-null   object        
 14  preciptotal      9601 non-null   float64

In [ ]:
#see the number of rows and columns of the combined dataframe
combined_df.shape

(9601, 23)

## spray_df

In [ ]:
#show sample output of the spray dataframe
spray_df.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [ ]:
# check unique value counts for time column
spray_df['Time'].value_counts()

7:44:32 PM     541
9:38:27 PM       5
9:40:27 PM       5
9:35:47 PM       5
8:55:26 PM       5
9:31:27 PM       5
8:59:06 PM       5
8:58:56 PM       5
8:57:56 PM       5
9:05:56 PM       5
8:57:46 PM       5
8:58:26 PM       5
8:55:36 PM       5
9:37:27 PM       5
8:55:46 PM       5
9:17:36 PM       4
8:29:36 PM       4
9:48:55 PM       4
8:25:11 PM       4
8:38:25 PM       4
9:33:37 PM       4
9:22:56 PM       4
8:55:06 PM       4
8:23:51 PM       4
9:36:27 PM       4
8:46:15 PM       4
8:31:46 PM       4
8:15:06 PM       4
8:34:06 PM       4
8:33:31 PM       4
8:25:16 PM       4
8:59:55 PM       4
8:24:51 PM       4
9:16:15 PM       4
8:24:11 PM       4
9:14:55 PM       4
9:18:35 PM       4
9:47:15 PM       4
9:42:25 PM       4
8:28:01 PM       4
9:22:16 PM       4
9:07:05 PM       4
9:14:15 PM       4
9:38:57 PM       4
9:10:46 PM       4
9:49:25 PM       4
8:58:06 PM       4
8:54:05 PM       4
8:40:16 PM       4
8:57:15 PM       4
8:20:11 PM       4
9:28:47 PM       4
8:51:35 PM  

As illustrated above, there is an absurd count of unique time at 7:44:32 PM.  To understand why, we dive deeper.

In [ ]:
#see the number of rows and columns of the spray dataframe
spray_df.shape

(14835, 4)

In [ ]:
# presence of null values in columns 
spray_df.isnull().value_counts()

Date   Time   Latitude  Longitude
False  False  False     False        14251
       True   False     False          584
dtype: int64

In [ ]:
# grouping null values by date
spray_df[spray_df['Time'].isnull()].groupby('Date').count()

,Time,Latitude,Longitude
Date,,,
2011-09-07,0,584,584


There seem to be duplicates of this particular date (2011-09-07) after it was grouped.  To make sure, we will do a duplicate check.

In [ ]:
# check rows with duplicate values in 'Date' column
spray_df[spray_df.duplicated(keep=False)]

,Date,Time,Latitude,Longitude
484,2011-09-07,7:43:40 PM,41.983917,-87.793088
485,2011-09-07,7:43:40 PM,41.983917,-87.793088
489,2011-09-07,7:44:32 PM,41.986460,-87.794225
490,2011-09-07,7:44:32 PM,41.986460,-87.794225
491,2011-09-07,7:44:32 PM,41.986460,-87.794225
492,2011-09-07,7:44:32 PM,41.986460,-87.794225
493,2011-09-07,7:44:32 PM,41.986460,-87.794225
494,2011-09-07,7:44:32 PM,41.986460,-87.794225
495,2011-09-07,7:44:32 PM,41.986460,-87.794225
496,2011-09-07,7:44:32 PM,41.986460,-87.794225


It seem our check was correct so we will remove all the duplicates within the spray dataframe to make sure we do not skew the results.

In [ ]:
spray_df.shape

(14835, 4)

In [ ]:
# drop all duplicates
spray_df = spray_df.drop_duplicates()

In [ ]:
# final number of rows
spray_df.shape

(14294, 4)

In [ ]:
#to make all columns consistant
# convert all column labels to lowercase
spray_df.columns = [clean_columns(col) for col in spray_df]

In [ ]:
#see sample output of the spray dtaframe
spray_df.head()

,date,time,latitude,longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [ ]:
#output to CSV file
train_df.to_csv('datasets/train_clean.csv', index = False)
weather_df.to_csv('datasets/weather_clean.csv', index = False)
spray_df.to_csv('datasets/spray_clean.csv', index = False)
combined_df.to_csv('datasets/combined_df.csv', index = False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=60b14a4f-b63c-4fa7-9c07-6649311f1d98' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>